# Introduction to Modules


This section begins our in-depth look at the Python *module*—the highest-level program
organization unit, which packages program code and data for reuse, and provides 
self-contained namespaces that minimize variable name clashes across your programs. In
concrete terms, modules typically correspond to Python program files. Each file is a
module, and modules import other modules to use the names they define. Modules
might also correspond to extensions coded in external languages such as C, Java, or
C#, and even to directories in package imports. Modules are processed with two statements and one important function:

- `import`  
    Lets a client (importer) fetch a module as a whole

- `from`  
    Allows clients to fetch particular names from a module
    
- `imp.reload` (`reload` in 2.X)  
    Provides a way to reload a module’s code without stopping Python

This first section reviews module 
basics, and offers a general look at the role of modules in overall program structure.
In the sections that follow, we’ll dig into the coding details behind the theory.

Along the way, we’ll flesh out module details omitted so far—you’ll learn about reloads,
the `__name__` and `__all__` attributes, package imports, relative import syntax, 3.3 
namespace packages, and so on. Because modules and classes are really just glorified *namespaces*, we’ll formalize namespace concepts here as well.

## Why Use Modules?


In short, modules provide an easy way to organize components into a system by serving
as self-contained packages of variables known as *namespaces*. All the names defined at
the top level of a module file become attributes of the imported module object. As we
saw in the last chapter of this book, imports give access to names in a module’s global
scope. That is, the module file’s global scope *morphs* into the module object’s attribute
namespace when it is imported. Ultimately, Python’s modules allow us to link individual 
files into a larger program system.

More specifically, modules have at least three roles:

- *Code reuse*  
    Modules let you save code in files permanently. Unlike
    code you type at the Python interactive prompt, which goes away when you exit
    Python, code in module files is *persistent*—it can be reloaded and rerun as many
    times as needed. Just as importantly, modules are a place to define names, known
    as *attributes*, which may be referenced by multiple external clients. When used
    well, this supports a *modular* program design that groups functionality into reusable 
    units.

- *System namespace partitioning*  
    Modules are also the highest-level program organization unit in Python. Although
    they are fundamentally just packages of names, these packages are also *self-contained*—
    you can never see a name in another file, unless you explicitly import that
    file. Much like the local scopes of functions, this helps avoid name clashes across
    your programs. In fact, you can’t avoid this feature—everything “lives” in a module, 
    both the code you run and the objects you create are always implicitly enclosed
    in modules. Because of that, modules are natural tools for grouping system components.

- *Implementing shared services or data*  
    From an operational perspective, modules are also useful for implementing components 
    that are shared across a system and hence require only a *single copy*. For
    instance, if you need to provide a global object that’s used by more than one function 
    or file, you can code it in a module that can then be imported by many clients.

At least that’s the abstract story—for you to truly understand the role of modules in a
Python system, we need to digress for a moment and explore the general structure of
a Python program.


## Python Program Architecture


So far in this book, I’ve sugarcoated some of the complexity in my descriptions of
Python programs. In practice, programs usually involve more than just one file. For all
but the simplest scripts, your programs will take the form of *multifile* systems—as the
code timing programs of the preceding chapter illustrate. Even if you can get by with
coding a single file yourself, you will almost certainly wind up using external files that
someone else has already written.

This section introduces the general *architecture* of Python programs—the way you divide 
a program into a collection of source files (a.k.a. modules) and link the parts into
a whole. As we’ll see, Python fosters a modular program structure that groups functionality 
into coherent and reusable units, in ways that are natural, and almost automatic. 
Along the way, we’ll also explore the central concepts of Python modules, imports, 
and object attributes.

### How to Structure a Program


At a base level, a Python program consists of text files containing Python *statements*,
with one main *top-level* file, and zero or more supplemental files known as *modules*.

Here’s how this works. The top-level (a.k.a. script) file contains the main flow of control
of your program—this is the file you run to launch your application. The module files
are libraries of tools used to collect components used by the top-level file, and possibly
elsewhere. Top-level files use tools defined in module files, and modules use tools 
defined in other modules.

Although they are files of code too, module files generally don’t do anything when run
directly; rather, they define tools intended for use in other files. A file *imports* a module
to gain access to the tools it defines, which are known as its *attributes*—variable names
attached to objects such as functions. Ultimately, we import modules and access their
attributes to use their tools.

### Imports and Attributes


Let’s make this a bit more concrete. [:numref:`program-architecture`]() sketches the structure of a Python
program composed of three files: `a.py`, `b.py`, and `c.py`. The file `a.py` is chosen to be the
top-level file; it will be a simple text file of statements, which is executed from top to
bottom when launched. The files `b.py` and `c.py` are modules; they are simple text files
of statements as well, but they are not usually launched directly. Instead, as explained
previously, modules are normally imported by other files that wish to use the tools the
modules define.

![Program architecture in Python. A program is a system of modules. It has one top-level script file (launched to run the program), and multiple module files (imported libraries of tools). Scripts and modules are both text files containing Python statements, though the statements in modules usually just create objects to be used later. Python’s standard library provides a collection of precoded modules.](../../imgs/program-architecture.jpg)
[:label:`program-architecture`]()

For instance, suppose the file `b.py` in [:numref:`program-architecture`]() defines a function called `spam`, for
external use. As we learned when studying functions in [Part IV](), `b.py` will contain a
Python `def` statement to generate the function, which you can later run by passing zero
or more values in parentheses after the function’s name:

```python
    def spam(text):                 # File b.py
        print(text, 'spam')
```

Now, suppose `a.py` wants to use `spam`. To this end, it might contain Python statements
such as the following:

```python
    import b                        # File a.py
    b.spam('gumby')                 # Prints "gumby spam"
```

The first of these, a Python `import` statement, gives the file `a.py` access to everything
defined by top-level code in the file `b.py`. The code import `b` roughly means:

Load the file `b.py` (unless it’s already loaded), and give me access to all its attributes
through the name `b`.

To satisfy such goals, `import` (and, as you’ll see later, `from`) statements execute and load
other files on request. More formally, in Python, cross-file module linking is not resolved 
until such `import` statements are executed at *runtime*; their net effect is to assign
module names—simple variables like `b`—to loaded module objects. In fact, the module
name used in an `import` statement serves two purposes: it identifies the external *file* to
be loaded, but it also becomes a *variable* assigned to the loaded module.

Similarly, objects *defined* by a module are also created at runtime, as the `import` is
executing: `import` literally runs statements in the target file one at a time to create its
contents. Along the way, every name assigned at the top-level of the file becomes an
attribute of the module, accessible to importers. For example, the second of the statements in `a.py` calls the function `spam` defined in the module `b`—created by running its
`def` statement during the import—using object attribute notation. The code `b.spam`
means:

Fetch the value of the name `spam` that lives within the object `b`.

This happens to be a callable function in our example, so we pass a string in parentheses
(`'gumby'`). If you actually type these files, save them, and run `a.py`, the words “gumby
spam” will be printed.

As we’ve seen, the `object.attribute` notation appears throughout Python code—most
objects have useful attributes that are fetched with the “.” operator. Some reference
callable objects like functions that take action (e.g., a salary computer), and others are
simple data values that denote more static objects and properties (e.g., a person’s
name).

The notion of importing is also completely general throughout Python. Any file can
import tools from any other file. For instance, the file `a.py` may import `b.py` to call its
function, but `b.py` might also import `c.py` to leverage different tools defined there. 
Import chains can go as deep as you like: in this example, the module `a` can import `b`,
which can import `c`, which can import `b` again, and so on.

Besides serving as the highest organizational structure, modules (and module packages,
described in [Chapter 24]()) are also the highest level of *code reuse* in Python. Coding
components in module files makes them useful in your original program, and in any
other programs you may write later. For instance, if after coding the program in [:numref:`program-architecture`]() we discover that the function `b.spam` is a general-purpose tool, we can reuse
it in a completely different program; all we have to do is import the file `b.py` again from
the other program’s files.

### Standard Library Modules


Notice the rightmost portion of [:numref:`program-architecture`](). Some of the modules that your programs
will import are provided by Python itself and are not files you will code.
Python automatically comes with a large collection of utility modules known as the
*standard library*. This collection, over 200 modules large at last count, contains 
platform-independent support for common programming tasks: operating system interfaces, 
object persistence, text pattern matching, network and Internet scripting, GUI
construction, and much more. None of these tools are part of the Python language
itself, but you can use them by importing the appropriate modules on any standard
Python installation. Because they are standard library modules, you can also be reasonably sure that they will be available and will work portably on most platforms on
which you will run Python.

This book’s examples employ a few of the standard library’s modules—`timeit`, `sys`,
and `os` in last section’s code, for instance—but we’ll really only scratch the surface of
the libraries story here. For a complete look, you should browse the standard Python
library reference manual, available either online at http://www.python.org, or with your
Python installation (via IDLE or Python’s Start button menu on some Windows). The
*PyDoc* tool discussed in [Chapter 15]() is another way to explore standard library modules.

Because there are so many modules, this is really the only way to get a feel for what
tools are available. You can also find tutorials on Python library tools in commercial
books that cover application-level programming, such as O’Reilly’s [Programming Python](http://www.oreilly.com/catalog/9780596158101), 
but the manuals are free, viewable in any web browser (in HTML format),
viewable in other formats (e.g., Windows help), and updated each time Python is re-released. See [Chapter 15]() for more pointers.

## How Imports Work


The prior section talked about importing modules without really explaining what happens 
when you do so. Because imports are at the heart of program structure in Python,
this section goes into more formal detail on the import operation to make this process
less abstract.

Some C programmers like to compare the Python module import operation to a C
`#include`, but they really shouldn’t—in Python, imports are not just textual insertions
of one file into another. They are really runtime operations that perform three distinct
steps the first time a program imports a given file:

1. *Find* the module’s file.
2. *Compile* it to byte code (if needed).
3. *Run* the module’s code to build the objects it defines.

To better understand module imports, we’ll explore these steps in turn. Bear in mind
that all three of these steps are carried out only the *first time* a module is imported
during a program’s execution; later imports of the same module in a program run
bypass all of these steps and simply fetch the already loaded module object in memory.
Technically, Python does this by storing loaded modules in a table named `sys.modules` 
and checking there at the start of an import operation. If the module is not present,
a three-step process begins.